In [ ]:
import numpy as np

In [13]:
!pip install nnfs

In [40]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

class Layer_dense:
  def __init__(self, inputs, neurons):
    self.w = 0.01*np.random.randn(inputs, neurons)
    self.b = np.zeros((1, neurons))

  def forward(self, inputs):
    self.output = np.dot(inputs, self.w) + self.b


class Relu:
  def forward(self, inputs):
    self.output = np.maximum(0, inputs)

class Softmax:
  def forward(self, inputs):
    exp = np.exp(inputs - np.max(inputs, axis=1, keepdims =True))

    probabilities = exp/np.sum(exp, axis= 1, keepdims=True)
    self.output = probabilities

  def backward(self, dvalues):
    # Create uninitialized array
    self.dinputs = np.empty_like(dvalues)
    # Enumerate outputs and gradients
    for index, (single_output, single_dvalues) in \
        enumerate(zip(self.output, dvalues)):
      # Flatten output array
      single_output = single_output.reshape(-1, 1)
      # Calculate Jacobian matrix of the output and
      jacobian_matrix = np.diagflat(single_output) - \
                        np.dot(single_output, single_output.T)
      # Calculate sample-wise gradient
      # and add it to the array of sample gradients
      self.dinputs[index] = np.dot(jacobian_matrix,
                                   single_dvalues)


class Loss:
  def calculate(self, output, y):
      sample_loss = self.forward(output, y)
      data_loss = np.mean(sample_loss)
      return data_loss
#_______________________________________________________________________________
class Catagorical_cross_entropy(Loss):
  def forward(self, y_pred, y_true):   # y_pred -> predicted value
                                         # y_true -> actiual value
      samples = len(y_pred)
      y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)


# pribalility of targey value
      if len(y_true.shape)==1:
         correct_confidences = y_pred_clipped[range(samples), y_true]

#     one-hor encoded lalbels
      elif len(y_true.shape)==2:
          correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
# losses
      negative_log_likelihoods = -np.log(correct_confidences)
      return negative_log_likelihoods

  def backward(self, dvalues, y_true):

    samples = len(dvalues)
    labels = len(dvalues[0])  # Number of labels/classes

    # If labels are sparse, turn them into one-hot vector
    if len(y_true.shape) == 1:
        y_true = np.eye(labels)[y_true]

    # Calculate gradient
    self.dinputs = -y_true / dvalues
    # Normalize gradient
    self.dinputs = self.dinputs / samples


#_______________________________________________________________________________
#_______________________________________________________________________________
class Softmax_loss_categorical_crossentropy():
  def __init__(self):
    self.activation = Softmax()
    self.loss = Catagorical_cross_entropy()

  def forward(self, inputs, y):
    self.activation.forward(inputs)
    self.outputs = self.activation.output # Changed from self.activation.outputs to self.activation.output

    return self.loss.calculate(self.outputs, y)

  def backward(self, dvalues, y):
    samples = len(dvalues)

    if len(y.shape)==2:
      y = np.argmax(y, axis =1)

    self.dinputs = dvalues.copy()
    self.dinputs[range(samples), y] -= 1
    self.dinputs = self.dinputs / samples

In [16]:
y= np.array([[1, 0, 0],
             [0, 0, 1],
             [0, 1, 0]])
np.argmax(y)

0

In [17]:
print(np.argmax(y, axis = 0))

[0 2 1]


In [39]:
softmax_outputs = np.array([[0.9, 0.8, 0.7],
                            [0.5, 0.4, 0.3],
                            [0.01, -0.7, -0.02]])
target = np.array([0, 1, 1])

softmax_loss = Softmax_loss_categorical_crossentropy()
softmax_loss.backward(softmax_outputs, target)
dvalues1 = softmax_loss.dinputs

activation = Softmax()
activation.output = softmax_outputs

loss = Catagorical_cross_entropy()
loss.backward(softmax_outputs, target) # Now you can call backward on loss
activation.backward(loss.dinputs)
dvalues2 = activation.dinputs

print(dvalues1)
print(dvalues2)

[[-0.03333333  0.26666667  0.23333333]
 [ 0.16666667 -0.2         0.1       ]
 [ 0.00333333 -0.56666667 -0.00666667]]
[[-0.03333333  0.26666668  0.23333333]
 [ 0.16666667 -0.2         0.09999999]
 [ 0.00333333 -0.56666666 -0.00666667]]
